# Load libraries

In [61]:
import sys
sys.path.insert(0, '/Users/jarad/fake_folder')

from jb_libraries import *
%matplotlib inline

import datetime as dt

#==========================================================================================

for_all_time = 'no'

if for_all_time == 'yes':
    date_start = '2015-11-10'
else:
    date_start = '2018-01-01'
    
date_end = '2018-01-31'
    
pretty_date = calendar.month_abbr[int(date_end[5:7])] + ' ' + date_end[:4]    

#==========================================================================================

title = 'Particle Report for month of ' + pretty_date + ' - v3'
#title = 'Particle report for all of time, from ' + date_start + ' to ' + date_end
#title = 'Particle Report for week ending ' + date_end

close_workbook = 'yes'

print(title)

Particle Report for month of Jan 2018 - v3


# Set Particle parts

In [32]:
# these are WORKING SKUS ONLY

particle_list = [3450,3451,3452,3453,3455,3454,3234,3233,2798,2725,2721,2723,2724,3051,3457,2799,3233,3234,2722]

part_check = pd.read_sql(
'''
SELECT
pd.part_id AS 'part id',
pd.products_name AS 'part name',
k.sku_mpn AS mpn,
k.sku_id AS 'sku id'
FROM products_description pd

LEFT JOIN
(SELECT
part_id,
sku_id,
sku_mpn
FROM skus
WHERE sku_status = 'working') k ON pd.part_id = k.part_id

WHERE pd.part_id IN '''+ str(tuple(particle_list)) +'''
ORDER BY pd.part_id
''', db)

first_dates = pd.read_sql(
'''
SELECT
MIN(DATE(o.date_purchased)) AS 'first date PN was purchased',
op.part_id AS 'part id'
FROM orders o
JOIN orders_products op ON o.orders_id = op.orders_id
WHERE op.part_id IN '''+ str(tuple(particle_list)) +'''
GROUP BY op.part_id
''', db)

particle_parts = pd.merge(part_check,
                         first_dates,
                         how = 'left',
                         on = 'part id')

print('%i duplicated part ids' % np.sum(particle_parts['part id'].duplicated()))
print('%i duplicated sku ids' % np.sum(particle_parts['sku id'].duplicated()))

0 duplicated part ids
0 duplicated sku ids


# Get Sales data

In [33]:
otb_main = pd.read_sql(
'''
SELECT
DATE(o.date_purchased) AS 'date purchased',
otb.orders_id AS 'orders id',
otb.sku_id AS 'sku id',
k.part_id AS 'part id',
IF(otb.barcode = '', 'no barcode', otb.barcode) AS barcode,
otb.qty AS 'otb qty bought'
FROM orders_to_barcodes otb
JOIN orders o ON otb.orders_id = o.orders_id
LEFT JOIN skus k ON otb.sku_id = k.sku_id
WHERE otb.sku_id IN '''+ str(tuple(particle_parts['sku id'])) +'''
''', db)

#=============================================================================================

op_main = pd.read_sql(
'''
SELECT
DATE(o.date_purchased) AS 'date purchased',
op.orders_id AS 'orders id',
op.part_id AS 'part id',
op.products_quantity AS 'qty bought'
FROM orders_products op
JOIN orders o ON op.orders_id = o.orders_id
WHERE op.part_id IN '''+ str(tuple(particle_parts['part id'])) +'''
''', db)

op_main['qty bought'] = op_main['qty bought'].astype(int)

#=============================================================================================

partials_main = pd.read_sql(
'''
SELECT
p1.pop_parent_id AS 'orders id',
DATE(o.date_purchased) AS 'date purchased',
p1.pop_partial_id AS 'partial id',
p1.pop_part_id AS 'part id',
p1.pop_quantity AS 'qty bought',
os.orders_status_name AS 'order status'

FROM partial_orders_products p1
JOIN partial_shipments p2 ON p1.pop_parent_id = p2.ps_parent_id 
AND p1.pop_partial_id = p2.ps_partial_id
JOIN orders_status os ON p2.ps_partial_status = os.orders_status_id
JOIN orders o ON p1.pop_parent_id = o.orders_id 

WHERE pop_part_id IN '''+ str(tuple(particle_parts['part id'])) +'''
''', db)

In [34]:
otb_expand = pd.DataFrame()

for index, row in otb_main.iterrows():
    mult = row['otb qty bought']
    inter_df = pd.DataFrame({'date purchased':[row['date purchased']] * mult,
                            'orders id':[row['orders id']] * mult,
                            'sku id':[row['sku id']] * mult,
                            'part id':[row['part id']] * mult,
                            'barcode':[row['barcode']] * mult,
                            'otb qty bought':[1] * mult})
    otb_expand = otb_expand.append(inter_df, ignore_index = True)
    
if otb_main['otb qty bought'].sum() == otb_expand['otb qty bought'].sum():
    print('match!')
else:
    print('mismatch!')    

match!


In [35]:
op_main['partial id'] = 1
op_main.drop(op_main[op_main['orders id'].isin(partials_main['orders id'])].index, inplace = True)

op_expand = pd.DataFrame()

for index, row in op_main.iterrows():
    mult = row['qty bought']
    inter_df = pd.DataFrame({'date purchased':[row['date purchased']] * mult,
                            'orders id':[row['orders id']] * mult,
                            'part id':[row['part id']] * mult,
                            'qty bought':[1] * mult,
                            'partial id':row['partial id'] * mult,
                            'table':['op'] * mult,
                            'partial id':row['partial id'] * mult})
    op_expand = op_expand.append(inter_df, ignore_index = True)
    
if op_main['qty bought'].sum() == op_expand['qty bought'].sum():
    print('match!')
else:
    print('mismatch!')    

match!


In [36]:
partials_expand = pd.DataFrame()

for index, row in partials_main.iterrows():
    mult = row['qty bought']
    inter_df = pd.DataFrame({'date purchased':[row['date purchased']] * mult,
                            'orders id':[row['orders id']] * mult,
                            'part id':[row['part id']] * mult,
                            'qty bought':[1] * mult,
                            'partial id':row['partial id'] * mult,
                            'table':['partials'] * mult,
                            'partial id':row['partial id'] * mult})
    partials_expand = partials_expand.append(inter_df, ignore_index = True)
    
if partials_main['qty bought'].sum() == partials_expand['qty bought'].sum():
    print('match!')
else:
    print('mismatch!')    

match!


In [37]:
sales_expand = pd.concat([op_expand, partials_expand])

In [38]:
otb_expand.sort_values(['orders id','part id'], inplace = True)
otb_expand = otb_expand.set_index(otb_expand.groupby(['orders id','part id']).cumcount(), append = True)
otb_expand.index.names = ['index','sub index']
otb_expand.index = otb_expand.index.droplevel(0)
otb_expand.reset_index(inplace = True)

#=====================================================================================

sales_expand.sort_values(['orders id','part id'], inplace = True)
sales_expand = sales_expand.set_index(sales_expand.groupby(['orders id','part id','table','partial id']).cumcount(), append = True)
sales_expand.index.names = ['index','sub index']
sales_expand.index = sales_expand.index.droplevel(0)
sales_expand.reset_index(inplace = True)

#=====================================================================================

main = pd.merge(sales_expand,
                otb_expand,
                how = 'left',
                on = ['date purchased','orders id','part id','sub index'])

#=====================================================================================

main['barcode'].fillna('no barcode', inplace = True)
main['otb qty bought'].fillna(0, inplace = True)

#=====================================================================================

def sku_fix(x):
    if pd.isnull(x['sku id']):
        sku = particle_parts['sku id'][particle_parts['part id'] == x['part id']].values[0]
        return int(sku)
    else:
        return int(x['sku id'])
    
main['sku id'] = main.apply(sku_fix, axis = 1)

#=====================================================================================

if main['qty bought'].sum() == sales_expand['qty bought'].sum():
    print('match!')
else:
    print('mismatch!')
    
#=====================================================================================    
    
if main['otb qty bought'].sum() == otb_main['otb qty bought'].sum():
    print('match!')
else:
    print('mismatch!')
    
#=====================================================================================    
    
main.drop('sub index',1,inplace = True)

#=====================================================================================    

main = main.groupby(['date purchased','orders id','part id','sku id','barcode','partial id','table'], as_index = False)[['qty bought']].sum()
main['date purchased'] = pd.to_datetime(main['date purchased'])

match!
match!


In [39]:
main.head()

,date purchased,orders id,part id,sku id,barcode,partial id,table,qty bought
0,2015-11-10,921416,2721,2973,no barcode,1,op,1
1,2015-11-10,921416,2725,2976,no barcode,1,op,1
2,2015-11-10,921419,2721,2973,no barcode,1,op,1
3,2015-11-10,921428,2721,2973,no barcode,1,op,1
4,2015-11-10,921428,2725,2976,no barcode,2,op,2


# Get other data

In [40]:
# all skus, not just working ones!!

skus = pd.read_sql(
'''
SELECT
part_id AS 'part id',
sku_id AS 'sku id',
sku_name AS name,
sku_mpn AS mpn
FROM skus
WHERE part_id IN '''+ str(tuple(particle_list)) +'''
''', db)

In [41]:
print('dupe sku id count:', np.sum(skus['sku id'].duplicated()))
print('dupe part id count:', np.sum(skus['part id'].duplicated()))

dupe sku id count: 0
dupe part id count: 3


In [42]:
ot_main = pd.read_sql(
'''
SELECT
*
FROM orders_total
WHERE orders_id IN '''+ str(tuple(main['orders id'])) +'''
''', db)

In [43]:
ot_main.columns = ot_main.columns.str.replace('_',' ')
ot_main['class'] = ot_main['class'].str.replace('ot_','')
ot = ot_main.groupby(['orders id','class'])[['value']].sum().unstack(1).fillna(0)
ot.columns = ot.columns.droplevel(0)
ot.reset_index(inplace = True)

cols = ['orders id','coupon','subtotal']

ot = ot[cols]
ot.columns = ['orders id','coupon amount','subtotal']

ot['coupon %'] = ot['coupon amount']/ot['subtotal']

In [44]:
parts = pd.read_sql(
'''
SELECT
part_id AS 'part id',
products_upc AS upc
FROM parts
WHERE part_id IN '''+ str(tuple(particle_list)) +'''
''', db)

In [45]:
orders = pd.read_sql(
'''
SELECT
DATE(o.date_purchased) AS 'date purchased',
o.orders_id AS 'orders id',
o.delivery_state AS 'delivery state',
o.delivery_country AS 'delivery country', 
o.delivery_postcode AS 'delivery postcode',
o.coupon_code AS 'coupon code', 

CASE
WHEN o.orders_reseller = 0 AND o.orders_super_reseller = 0 THEN 'non reseller'
WHEN o.orders_reseller = 1 AND o.orders_super_reseller = 0 THEN 'reseller'
WHEN o.orders_super_reseller = 1 THEN 'super reseller'
ELSE 'you missed one'
END AS 'customer type',
LOWER(os.orders_status_name) AS 'orders status'

FROM orders o
JOIN orders_status os ON o.orders_status = os.orders_status_id
WHERE orders_id IN '''+ str(tuple(main['orders id'])) +'''
''', db)

orders['customer type'].value_counts()

non reseller      4760
reseller           445
super reseller      22
Name: customer type, dtype: int64

In [46]:
op = pd.read_sql(
'''
SELECT
orders_id AS 'orders id',
part_id AS 'part id',
products_price AS 'price'
FROM orders_products
WHERE orders_id IN '''+ str(tuple(main['orders id'])) +'''
AND part_id IN '''+ str(tuple(particle_list)) +'''
''', db)

In [47]:
together = pd.merge(main,
                    skus,
                    how = 'left',
                    on = ['part id','sku id']).merge(ot,
                                                    how = 'left',
                                                    on = 'orders id').merge(parts,
                                                                           how = 'left',
                                                                           on = 'part id').merge(orders.drop('date purchased',1),
                                                                                                how = 'left',
                                                                                                on = 'orders id').merge(op,
                                                                                                                        how = 'left',
                                                                                                                        on = ['orders id','part id'])

if (together['qty bought'].sum() == op_main['qty bought'].sum() + partials_main['qty bought'].sum() == main['qty bought'].sum()).all():
    print('match!')
else:
    print('mismatch!')
    print(together['qty bought'].sum(), op_main['qty bought'].sum(), main['qty bought'].sum())

match!


In [48]:
together = together[['part id',
                      'sku id',
                      'name',
                      'mpn',
                      'price',
                      'orders id',
                      'barcode',
                      'qty bought',
                      'subtotal',
                      'coupon amount',
                      'coupon %',
                      'date purchased',
                      'delivery state',
                      'delivery country',
                      'delivery postcode',
                      'coupon code',
                      'customer type',
                      'upc']]

together['part total after coupon'] = together['price'] * (1 - together['coupon %'])
together['amount saved with coupon'] = together['price'] - together['part total after coupon']
together['date purchased'] = pd.to_datetime(together['date purchased'])

if (together['qty bought'].sum() == op_main['qty bought'].sum() + partials_main['qty bought'].sum() == main['qty bought'].sum()).all():
    print('match!')
else:
    print('mismatch!')
    print(together['qty bought'].sum(), op_main['qty bought'].sum(), main['qty bought'].sum())

match!


# Get returns

In [49]:
ret1 = pd.read_sql(
'''
SELECT
DATE(rq_date) AS 'date returned',
rq_orders_id AS 'orders id',
part_id AS 'part id',
rq_quantity AS 'qty returned',
'unknown' AS 'return type',
'np.nan' AS 'qty bought',
'np.nan' AS 'qty available',
'unknown' AS action,
'np.nan' AS 'amount refunded',
'np.nan' AS 'amount credited',
'np.nan' AS 'amount cancelled'
FROM return_quantities
WHERE part_id IN '''+ str(tuple(particle_parts['part id'])) +'''
AND rq_quantity > 0
''', db)

#==========================================================================================

ret1['sku id'] = ret1['part id'].map(dict(zip(particle_parts['part id'], particle_parts['sku id'])))
ret1.replace('np.nan',np.nan,inplace = True)

#==========================================================================================

ret2 = pd.read_sql(
'''
SELECT
DATE(timestamp) AS 'date returned',
orders_id AS 'orders id',
part_id AS 'part id',
sku_id AS 'sku id',
issue_type AS 'return type',
qty_ordered AS 'qty bought',
qty_available AS 'qty available',
action,
amount_refunded AS 'amount refunded',
store_credit AS 'amount credited',
amount_canceled AS 'amount cancelled'
FROM cs_error_reports
WHERE sku_id IN '''+ str(tuple(particle_parts['sku id'])) +'''
OR part_id IN '''+ str(tuple(particle_parts['part id'])) +'''
''', db)

ret2['return type'] = ret2['return type'].str.replace('_',' ')

#==========================================================================================

returns = pd.concat([ret1, ret2])
returns['name'] = returns['sku id'].map(dict(zip(skus['sku id'], skus['name'])))
print('%i dupes' % np.sum(returns.duplicated()))

0 dupes


In [50]:
cols = ['part id',
       'sku id',
       'name',
       'orders id',
       'date returned',
       'return type',
       'action',
       'qty available',
       'qty bought',
       'qty returned',
       'amount cancelled',
       'amount credited',
       'amount refunded']

if len(returns.columns) == len(cols):
    returns = returns[cols]
    print('match!')
else:
    print('mismatch!')
    print(set(list(returns.columns)) - set(cols))
    
returns['date returned'] = pd.to_datetime(returns['date returned'])    
returns = returns[returns['date returned'].between(date_start, date_end)]

returns.fillna(0, inplace = True)

match!


# Check

In [51]:
op_check = pd.read_sql(
'''
SELECT
orders_id AS 'orders id',
part_id AS 'part id',
products_quantity AS 'op qty'
FROM orders_products
WHERE part_id IN '''+ str(tuple(particle_list)) +'''
''', db)

if (np.sum(op_check['op qty']) == main['qty bought'].sum() == together['qty bought'].sum()).all():
    print('match!')
else:
    print('mismatch!')

match!


# Restrict

In [52]:
if for_all_time == 'no':
    
    together = together[together['date purchased'].between(date_start, date_end)]

cols = ['part id',
       'sku id',
       'name',
       'mpn',
       'price',
       'orders id',
       'date purchased',
       'customer type',
       'barcode',
       'upc',
       'qty bought',
       'subtotal',
       'coupon code',
       'coupon amount',
       'coupon %',
       'part total after coupon',
       'amount saved with coupon',
       'delivery state',
       'delivery country',
       'delivery postcode']

if len(together.columns) == len(cols):
    together = together[cols]
    print('match!')
else:
    print('mismatch!')
    print(set(list(together.columns)) - set(cols))

match!


# Get stock

In [53]:
stock_main = pd.read_sql(
'''
SELECT
sku_id AS 'sku id',
timestamp,
DATE(timestamp) AS 'stock date',
stl_qty_new AS 'stock'
FROM stock_log
WHERE sku_id IN '''+ str(tuple(particle_parts['sku id'])) +'''
''', db)

stock_main.sort_values(['sku id','timestamp'], ascending = [True,False], inplace = True)
stock_main.drop_duplicates('sku id', keep = 'first', inplace = True)
stock_main.drop('timestamp',1,inplace = True)

In [54]:
os = pd.merge(orders[['date purchased','orders id','orders status','customer type']],
              main[['orders id','qty bought','part id','sku id']],
              how = 'right',
              on = 'orders id')

#==========================================================================================

ons = os[os['orders status'].isin(['printed','pending'])].groupby(['sku id','customer type'])[['qty bought']].sum().unstack(1).fillna(0)
ons.columns = ons.columns.droplevel(0)

ls = ['non reseller','reseller','super reseller']

for c in ls:
    if c not in ons.columns:
        ons[c] = 0        
        
ons.rename(columns = {'non reseller':'non reseller ONS',
                      'reseller':'reseller ONS',
                      'super reseller':'super reseller ONS'}, inplace = True)
ons.reset_index(inplace = True)

stock = pd.merge(stock_main,
                ons,
                how = 'left',
                on = 'sku id').merge(skus[['sku id','part id','mpn','name']],
                                    how = 'left',
                                    on = 'sku id').merge(main[main['date purchased'].between(date_start, date_end)].groupby('sku id', as_index = False)[['qty bought']].sum(),
                                                        how = 'left',
                                                        on = 'sku id').fillna(0)
stock.drop('qty bought', 1, inplace = True)

stock['effective stock'] = stock['stock'] - stock[['non reseller ONS','reseller ONS','super reseller ONS']].sum(1)
    
#==========================================================================================    
    
if len(stock['sku id'].unique()) == len(main['sku id'].unique()):
    print('match!')
else:
    print('mismatch!')    
    
cols = ['part id',
        'sku id',
        'name',
        'mpn',
        'stock',
        'non reseller ONS',
        'reseller ONS',
        'super reseller ONS',
        'effective stock']

stock = stock[cols]    

match!


# Excel Start

In [62]:
returns['date returned'] = [str(x) for x in returns['date returned']]
together['date purchased'] = [str(x)[:11] for x in together['date purchased']]

if close_workbook == 'yes':

#    writer = pd.ExcelWriter(title + '.xlsx', engine = 'xlsxwriter')

    sheets = ['Retail Returns',
             'Retail Sales by Serial Number',
             'Retail Sales by Qty',
             'Stock',
             'Distributor by Serial Number',
             'Distributor by Qty']

    workbook = xlsxwriter.Workbook(title + '.xlsx', {'nan_inf_to_errors': True})
    
#    workbook = writer.book
    for sheet in sheets:
        workbook.add_worksheet(sheet)

    my_worksheets = {}
    for worksheet in workbook.worksheets():
        my_worksheets[worksheet.get_name()] = worksheet    

    for k,v in my_worksheets.items():
        worksheet = my_worksheets[k]
        worksheet.write(0, 0, title, workbook.add_format({'font_size':20,
                                                      'font_name':'Arial (Bold)'}))
        worksheet.write(1, 0, k, workbook.add_format({'font_size':20,
                                                      'font_name':'Arial (Bold)'}))

    money = ['price','subtotal',
             'coupon amount',
             'part total after coupon',
             'amount saved with coupon',
             'amount refunded',
             'amount credited',
             'amount cancelled']
    percent = ['coupon %']
    date = ['date']    

#===== Retail Returns
#===================================================================================================================

    worksheet = my_worksheets['Retail Returns']
    if returns.empty:
        worksheet.write(3, 0, 'There were no returns', workbook.add_format({'font_size':20,
                                                                            'font_name':'Arial (Bold)'}))
    else:
        start_row = 3
        start_col = 0
        for c in range(len(returns.columns)):
            worksheet.write(start_row, start_col + c, returns.columns[c].title(), workbook.add_format({'top':1,
                                                                                   'bottom':1,
                                                                                   'left':1,
                                                                                   'right':1,
                                                                                   'valign':'center',
                                                                                   'align':'center'}))
            for i in range(len(returns)):
                if returns.columns[c] in money:
                    fmt = workbook.add_format({'num_format':'$#,##0.00'})
                else:
                    fmt = None

                worksheet.write(start_row + 1 + i, start_col + c, returns.iloc[i,c], fmt)
                worksheet.set_column(start_col, start_col + len(returns.columns), 25)

#===== Retail Sales by Serial Number     
#===================================================================================================================

    retail_sales = together.copy()
    retail_sales = retail_sales[retail_sales['customer type'] == 'non reseller']

    worksheet = my_worksheets['Retail Sales by Serial Number']

    start_row = 3
    start_col = 0   
    for c in range(len(retail_sales.columns)):

        # headers
        worksheet.write(start_row, start_col + c, retail_sales.columns[c].title(), workbook.add_format({'top':1,
                                                                                               'bottom':1,
                                                                                               'left':1,
                                                                                               'right':1,
                                                                                               'valign':'center',
                                                                                               'align':'center'})) 
        # set column
        worksheet.set_column(start_col + c, start_col + c, 25)

        # data with conditional format
        for i in range(len(retail_sales)):

            if retail_sales.columns[c] in money:
                fmt = workbook.add_format({'num_format':'$#,##0.00'})
            elif retail_sales.columns[c] in percent:
                fmt = workbook.add_format({'num_format':'0.00%'})
            elif retail_sales.columns[c] in date:
                fmt = workbook.add_format({'num_format':'yyyy-mm-dd'})        
            else:
                fmt = None
            worksheet.write(start_row + 1 + i, start_col + c, retail_sales.iloc[i,c], fmt) 

#===== Retail Sales by Qty     
#===================================================================================================================

    retail_by_qty = retail_sales.groupby(['part id','name','mpn','upc'], as_index = False)[['qty bought']].sum()

    worksheet = my_worksheets['Retail Sales by Qty']

    start_row = 3
    start_col = 0   
    for c in range(len(retail_by_qty.columns)):

        # headers
        worksheet.write(start_row, start_col + c, retail_by_qty.columns[c].title(), workbook.add_format({'top':1,
                                                                                               'bottom':1,
                                                                                               'left':1,
                                                                                               'right':1,
                                                                                               'valign':'center',
                                                                                               'align':'center'})) 
        # set column
        worksheet.set_column(start_col + c, start_col + c, 25)

        # data with conditional format
        for i in range(len(retail_by_qty)):

            if retail_by_qty.columns[c] == 'qty bought':
                fmt = workbook.add_format({'num_format':'#,##0'})
            else:
                fmt = None
            worksheet.write(start_row + 1 + i, start_col + c, retail_by_qty.iloc[i,c], fmt) 

#===== Distributor Sales by Serial Number     
#===================================================================================================================

    dist_sales = together.copy()
    dist_sales = dist_sales[dist_sales['customer type'].isin(['reseller','super reseller'])]

    worksheet = my_worksheets['Distributor by Serial Number']

    start_row = 3
    start_col = 0   
    for c in range(len(dist_sales.columns)):

        # headers
        worksheet.write(start_row, start_col + c, dist_sales.columns[c].title(), workbook.add_format({'top':1,
                                                                                               'bottom':1,
                                                                                               'left':1,
                                                                                               'right':1,
                                                                                               'valign':'center',
                                                                                               'align':'center'})) 
        # set column
        worksheet.set_column(start_col + c, start_col + c, 25)

        # data with conditional format
        for i in range(len(dist_sales)):

            if dist_sales.columns[c] in money:
                fmt = workbook.add_format({'num_format':'$#,##0.00'})
            elif dist_sales.columns[c] in percent:
                fmt = workbook.add_format({'num_format':'0.00%'})
            elif dist_sales.columns[c] in date:
                fmt = workbook.add_format({'num_format':'yyyy-mm-dd'})
            else:
                fmt = None
            worksheet.write(start_row + 1 + i, start_col + c, dist_sales.iloc[i,c], fmt) 

#===== Distributor Sales by Qty     
#===================================================================================================================

    dist_by_qty = dist_sales.groupby(['part id','name','mpn','upc'], as_index = False)[['qty bought']].sum()

    worksheet = my_worksheets['Distributor by Qty']

    start_row = 3
    start_col = 0   
    for c in range(len(dist_by_qty.columns)):

        # headers
        worksheet.write(start_row, start_col + c, dist_by_qty.columns[c].title(), workbook.add_format({'top':1,
                                                                                               'bottom':1,
                                                                                               'left':1,
                                                                                               'right':1,
                                                                                               'valign':'center',
                                                                                               'align':'center'})) 
        # set column
        worksheet.set_column(start_col + c, start_col + c, 25)

        # data with conditional format
        for i in range(len(dist_by_qty)):

            if dist_by_qty.columns[c] == 'qty bought':
                fmt = workbook.add_format({'num_format':'#,##0'})
            else:
                fmt = None
            worksheet.write(start_row + 1 + i, start_col + c, dist_by_qty.iloc[i,c], fmt)         

#===== Stock
#===================================================================================================================

    worksheet = my_worksheets['Stock']

    start_row = 3
    start_col = 0   
    for c in range(len(stock.columns)):

        # headers
        worksheet.write(start_row, start_col + c, stock.columns[c].title(), workbook.add_format({'top':1,
                                                                                               'bottom':1,
                                                                                               'left':1,
                                                                                               'right':1,
                                                                                               'valign':'center',
                                                                                               'align':'center'})) 
        # set column
        worksheet.set_column(start_col + c, start_col + c, 25)

        # data with conditional format
        for i in range(len(stock)):

            if stock.columns[c] in ('qty bought','stock'):
                fmt = workbook.add_format({'num_format':'#,##0'})
            else:
                fmt = None
            worksheet.write(start_row + 1 + i, start_col + c, stock.iloc[i,c], fmt)   

#===== Close Workbook
#===================================================================================================================

    workbook.close()

In [63]:
print('done')    

done


In [64]:
title

'Particle Report for month of Jan 2018 - v3'